<a href="https://colab.research.google.com/github/biodatlab/nbdt-llm/blob/main/Fetch_Recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to access and fetch recommendations from the database

## Setup

In [2]:
!pip install numpy==1.23.2
!pip install transformers==4.28.0
!pip install -U sentence-transformers
!pip install datasets
!pip install langchain
!pip install torch
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install tensorflow
!pip install openai
!pip install -qU pinecone-client[grpc]
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=65e42fc45ab5a690b797db873d0f6f7a6b6d598b0ce8321694e79c8c18644cdc
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

In [3]:
import os
import shutil

import numpy as np
import pandas as pd
# import openai
import pinecone
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("en_core_web_sm")

# openai.api_key = ''
PINECONE_API_KEY = 'c590f8d4-d335-4ccc-9743-fd90ab8905b8'
PINECONE_ENV = 'us-west4-gcp-free'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
    )

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
!gdown 1-JiQjBZXUNsFoR0SQ8CQTZTKJMM1bsys
!gdown 1--NNLlqTWx6zSRdhDN_yjPrht4vEhHVH
!gdown 1NfMFVYC9MFFF1brutfvjUsoqC93kD_Li

Downloading...
From: https://drive.google.com/uc?id=1-JiQjBZXUNsFoR0SQ8CQTZTKJMM1bsys
To: /content/bioarxiv.csv
100% 54.3M/54.3M [00:00<00:00, 133MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--NNLlqTWx6zSRdhDN_yjPrht4vEhHVH
To: /content/PLOS_ONE2.csv
100% 57.6M/57.6M [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NfMFVYC9MFFF1brutfvjUsoqC93kD_Li
To: /content/arxiv2.csv
100% 5.59M/5.59M [00:00<00:00, 166MB/s]


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
index_name = 'reviewer-assignment'
index = pinecone.GRPCIndex(index_name)

## Helper Functions

In [7]:
from transformers import LongformerTokenizer, LongformerModel
import torch

def create_lbert_embed(sents,bundle):
  tokenizer = bundle[0]
  model = bundle[1]
  model.cpu()
  tokens = tokenizer(sents,padding=True,truncation=True,return_tensors='pt')
  device = torch.device('cpu')
  tokens = tokens.to(device)
  with torch.no_grad():
    embeds = model(**tokens, output_hidden_states=True,return_dict=True).pooler_output
  return embeds.cpu()

In [8]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
lbert_bundle = (tokenizer,model)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def get_matches(query,k=3,include_metadata=True):
  """
  Queries the index to get matches from the database and prints them.

          Parameters:
                query (int) : containing the abstract
                k (int) : fetches best 'k' results
                include_metadata (bool) : fetches the title and abstract of the match as well

          Returns:
                matches : The actual object returned by the index.query() method, if you require any additional data about the matches.
  """
  encoded_query = create_lbert_embed(query,lbert_bundle)
  matches = index.query(encoded_query.tolist()[0],top_k=k,include_metadata=include_metadata)
  for i,match in enumerate(matches['matches']):
    print(f"Match {i+1}")
    print(f"id : {match['id']}")
    print(f"score : {match['score']}")
    print(f"title : {match['metadata']['identifier']}")
    print(f"abstract: {match['metadata']['abstract']}")
    print('----------------------------------------------------------------------------------------------------------------------------')
  return matches

## Create queries

In [9]:
bioarx = pd.read_csv('bioarxiv.csv')

In [10]:
bioarx = bioarx[['title','abstract']]
bioarx = bioarx.rename(columns={'title':'identifier'})
bioarx = bioarx.to_dict('records')

{'identifier': 'Estimating the prevalence of missing experiments in a neuroimaging meta-analysis',
 'abstract': 'Coordinate-based meta-analyses (CBMA) allow researchers to combine the results from multiple fMRI experiments with the goal of obtaining results that are more likely to generalise. However, the interpretation of CBMA findings can be impaired by the file drawer problem, a type of publications bias that refers to experiments that are carried out but are not published. Using foci per contrast count data from the BrainMap database, we propose a zero-truncated modelling approach that allows us to estimate the prevalence of non-significant experiments. We validate our method with simulations and real coordinate data generated from the Human Connectome Project. Application of our method to the data from BrainMap provides evidence for the existence of a file drawer effect, with the rate of missing experiments estimated as at least 6 per 100 reported.'}

In [19]:
query = bioarx[43]['abstract']

In [13]:
# query = '' # Any piece of text

In [25]:
matches = get_matches(query)

Match 1
id : bioarx-43
score : 1.0
title : Automation of citation screening in pre-clinical systematic reviews
abstract: BackgroundThe amount of published in vivo studies and the speed researchers are publishing them make it virtually impossible to follow the recent development in the field. Systematic review emerged as a method to summarise and analyse the studies quantitatively and critically but it is often out-of-date due to its lengthy process.

MethodWe invited five machine learning and text-mining groups to build classifiers for identifying publications relevant to neuropathic pain (33814 training publications). We kept 1188 publications for the assessment of the performance of different classifiers. Two groups participated in the next stage: testing their algorithm on datasets labeled for psychosis (11777/2944) and datasets labeled for Vitamin D in multiple sclerosis (train/text: 2038/510).

ResultThe performances (sensitive/specificity) of the most promising classifier built f